# <p style="text-align: center;"> Final Project - Introduction to Data Science</p>
# <p style="text-align: center;"> <b> Data Modeling </b></p>
---

## Member Information
| Name              | ID       |
|-------------------|----------|
|Tran Dinh Quang    | 21127406 |
|Nguyen Hong Hanh   | 21127503 |
|Do Quoc Tri  | 21127556 |
| Nguyen Khanh Nhan | 21127657         |


## Table of contents
- [0. Import libraries](#0-import-libraries)
- [1. Problem statement](#1-problem-statement)
    - [1.1 State the problem](#11-state-the-problem)
    - [1.2 Purpose of solving the problem](#12-purpose-of-solving-the-problem)
- [2. Data preparation](#2-data-preparation)
    - [2.1 Pre-processing](#21-pre-processing)
    - [2.2 Splitting data](#22-splitting-data)
- [3. Data modeling](#3-data-modeling)
    - [3.1 Model selection](#31-model-selection)
    - [3.2 Fine-tuning model](#32-fine-tuning-model)
    - [3.3 Model training](#33-model-training)
    - [3.4 Basic ensemble method](#34-basic-ensemble-method)
    - [3.5 Model evaluation](#35-model-evaluation)


---

## 0. Import libraries

## 1. Problem Statement

### 1.1 State the problem

### 1.2 Purpose of solving the problem

## 2. Data preparation

### 2.1 Pre-processing

### 2.2 Splitting data

## 3. Data Modeling

### 3.1 Model selection

### 3.2 Fine-tuning model

### 3.3 Model training

### 3.4 Basic ensemble method

### 3.5 Model evaluation